## Creating and solving your Application 1 with Gurobi

### Restaurant Capacity Maximization Problem



**Decision Variables**

$f_i$ : 1 if table i  is used, 0 otherwise, where $i=1...N$

$g_{ij}:$ 1 if both table i and j are used, 0 otherwise,  

$x_i =$ bottom left x coordinate of table i

$y_i =$ bottom left x coordinate of table i

$l_{ij} =$ 1 if table i is located left to j

$b_{ij} =$ 1 if table i is located below j


**Parameters**

$d$ : The minimum physical distance required between tables

$c_i =$ max # people that can be seated @ table i, where $i=1...N$

$C$ = Minimum desired establishment capacity

$w_i =$ width of table i

$h_i =$ height of table i

$R_w =$ width of dining space

$R_h =$ height of dining space

$U_{ij}$ = Underperformance measure of staff when table i and j are used


**Objective Function**

$min$ $z$ = $g_1^+ + g_2^- +\sum_{i=1}^{N}\sum_{j=1, i\neq j}^{N}g_{ij}U_{ij}$


**Constraints**

**s.t**
Force non-linear element to be 0 or 1:

$g_{ij} >= f_i+f_j-1 \ i, j \in T and \ i \neq j$

No overlap if the following constraint is true:

$l_{ij} + l_{ji} + b_{ij} + b_{ji} + (1-x_i) + (1-x_j) >= 1, \ i, j \in T, \ i<j$ 

If a table i is to the left of table j, table i's right edge x coordinate  must be less than table j's x left edge x coordinate:

$x_i +w_i <= x_j + W - Wl_{ij}, \ i, j \in T$

If table i is below j, table i's top edge y coordinate must be less than table j's bottom edge y coordinate:

$y_i +h_i <= y_j + H - Hb_{ij}, \ i, j \in T$

Tables must exist within the dining space:

$x_i >= 0, \ i \in T$

$x_i+w_i <= R_x, \ i \in T$

$y_i >= 0, \ i \in T$

$y_i+h_i <= R_h, \ i \in T$

Unused table space must be under U:

$R_wR_h \ - \sum_{i=1}^{N}f_i(w_i)(h_i) - (g_1^+-g_1^-) = U, \ i \in T$ 

Facility capacity must be over C to remain profitable

$\sum_{i=1}^{N}c_if_i - (g_2^+-g_2^-) = C, \ i \in T$

Binary variable constraints:

$l_{ij}, b_{ij} \ \in {0,1}$

$f_{i} \ \in {0,1}$

$g_{ij}\in {0,1}$

#### Step 1: Import grobipy module

In [509]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
from gurobipy import quicksum
import plotly.graph_objects as go

#### Step 2: Define the model

In [510]:
m = gp.Model()

#### Step 3: Define the sets and sizes

#### Step 4: Define the parameters

In [512]:

    #number of tables
    N = 3
    
    #Minimum physical distane required between tables
    d = 2
    
    #capacity for each table
    c = [2, 2, 2]
    
    #width for each table
    w = [1, 1, 2]
   
    #account for physical distancing margin
    for i in range(len(w)):
        w[i] += d

    #height for each table
    h = [1, 2, 2]
    
    #account for physical distancing margin
    for i in range(len(h)):
        h[i] += d
    
    #Width of dining space
    Rw = 7

    #Height of dining space
    Rh = 7
    
    #Unused table space goal
    U = 2

    #Minimum desired establishment capacity goal
    C = 10

    #Underperformance measure of staff when table i and j are used Uij[rows][columns]
    Uij =  [[1,1,1],
            [1,1,1],
            [1,1,1]]

#### Step 5: Define decision variables

In [513]:
#binary, if table i is used
f = {}
for i in range(N):
    f[i] = m.addVar(vtype=GRB.BINARY, name="f_" +str(i))

#bottom left x coordinate of table i
x = {}
for i in range(N):
    x[i] = m.addVar(vtype=GRB.CONTINUOUS, name="x_"+str(i))
    
#bottom left y coordinate of table i
y = {}
for i in range(N):
    y[i] = m.addVar(vtype=GRB.CONTINUOUS, name="y_"+str(i))
    
    
#binary, if table i is located left to table j
l = {}
for i in range(N):
    for j in range(N):
        if i != j:
            l[i, j] = m.addVar(vtype=GRB.BINARY, name="l_" +str(i)+str(j))

#binary, if table i is located below to table j
b = {}
for i in range(N):
    for j in range(N):
        if i != j:
            b[i, j] = m.addVar(vtype=GRB.BINARY, name="b_" +str(i)+str(j))

#binary, if table i and table j are both used
g = {}
for i in range(N):
    for j in range(N):
        if i != j:
            g[i, j] = m.addVar(vtype=GRB.BINARY, name="g_" +str(i)+str(j))

#surplus for goal 1  
g1={}
g1 = m.addVar(vtype=GRB.CONTINUOUS, name="g1")

#slack/surplus for goal 2
g2={}
g2= m.addVar(vtype=GRB.CONTINUOUS, name="g2")

#### Step 6: Set the objective function

In [514]:
nonlinearobj = quicksum((g[i, j]*Uij[i][j])/2 for i in range(N) for j in range(N) if i != j)
m.setObjective(g1+g2+(1/2*nonlinearobj), GRB.MINIMIZE)

#### Step 7: Add constraints

In [515]:
# no overlap
m.addConstrs(l[i,j] + l[j,i] + b[i,j] + b[j,i] + (1-f[i]) + (1-f[j])  >= 1 for i in range(N) for j in range(N) if i != j)

# If a table i is to the left of table j, table i's right edge x coordinate  must be less than table j's x left edge x coordinate
m.addConstrs(x[i] + w[i] <= (x[j] + Rw - Rw*l[i,j]) for i in range(N) for j in range(N) if i != j)

#If table i is below j, table i's top edge y coordinate must be less than table j's bottom edge y coordinate
m.addConstrs(y[i] + h[i] <= (y[j] + Rh - Rh*b[i,j]) for i in range(N) for j in range(N) if i != j)

#table space must exist within the dining space
m.addConstrs(x[i] >= 0 for i in range(N))
m.addConstrs(x[i]+w[i]<= Rw for i in range (N))
m.addConstrs(y[i] >= 0 for i in range(N))
m.addConstrs(y[i]+h[i]<= Rh for i in range (N))

#Unused table space must be under U - GOAL 1
m.addConstrs((Rw*Rh) - quicksum((w[i])*(h[i])*f[i] for i in range(N)) - g1 == U for i in range(N))

#Facility capacity must be over C to remain profitable - GOAL 2
m.addConstrs(quicksum(f[i]*c[i] for i in range(N)) + g2 == C for i in range(N))

#Quadratic constraint
m.addConstrs(g[i,j] >= f[i] + f[j] - 1 for i in range(N) for j in range(N) if i != j)

#update model
m.update()

#### Step 8: Solve the model

In [516]:
m.optimize()

Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (win64)
Optimize a model with 42 rows, 29 columns and 126 nonzeros
Model fingerprint: 0x3d5447a8
Variable types: 8 continuous, 21 integer (21 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [3e-01, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+01]
Found heuristic solution: objective 39.0000000
Presolve removed 26 rows and 9 columns
Presolve time: 0.00s
Presolved: 16 rows, 20 columns, 53 nonzeros
Variable types: 3 continuous, 17 integer (17 binary)

Root relaxation: objective 1.550000e+01, 6 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   15.50000    0    6   39.00000   15.50000  60.3%     -    0s
H    0     0                      28.5000000   15.50000  45.6%     -    0s
H    0     0                      15.5000000   15.50000  0.00%     -

#### Insert variable values

In [517]:
#Goal 1 print
print("Goal 1: Unused space must be under", U, "m^2.", )
print("Goal 1 Result: Unused space was", g1.x, "m^2.", )
#Goal 2 print
print("Goal 2: Facility capacity must be over", C, "to remain profitable.", )
print("Goal 2 Result: Facility capacity was unmet by", g2.x, ".", )
#nonlinear element print
print("Underperformance level for this table setup is", nonlinearobj.getValue())
tableUsedData = []
for tableUsed in f:
    tableUsedData.append(f[tableUsed].x)
xCoords = []
for i in x:
    xCoords.append(x[i].x)
yCoords = []
for i in y:
    yCoords.append(y[i].x)
d = {'ifUsed' : tableUsedData, 'x-coord' : xCoords, 'y-coord' : yCoords, 'capacity' : c, 'table space width' : w, 'table space height' : h}
df = pd.DataFrame(data=d)
df



Goal 1: Unused space must be under 2 m^2.
Goal 1 Result: Unused space was 10.0 m^2.
Goal 2: Facility capacity must be over 10 to remain profitable.
Goal 2 Result: Facility capacity was unmet by 4.0 .
Underperformance level for this table setup is 3.0


,ifUsed,x-coord,y-coord,capacity,table space width,table space height
0,1.0,0.0,0.0,2,3,3
1,1.0,4.0,0.0,2,3,4
2,1.0,0.0,3.0,2,4,4


In [518]:
xCoordPlot = []
yCoordPlot = []
for i in f:
    if(f[i].x == 1):
        #top border
        xCoordPlot.append(0)
        xCoordPlot.append(Rw)
        yCoordPlot.append(Rh)
        yCoordPlot.append(Rh)
        xCoordPlot.append("none")
        yCoordPlot.append("none")
        
        #right border
        xCoordPlot.append(Rw)
        xCoordPlot.append(Rw)
        yCoordPlot.append(0)
        yCoordPlot.append(Rh)
        xCoordPlot.append("none")
        yCoordPlot.append("none")
        #x
        xCoordPlot.append(x[i].x)
        xCoordPlot.append(x[i].x)
        xCoordPlot.append(x[i].x + w[i])
        xCoordPlot.append(x[i].x + w[i])
        xCoordPlot.append(x[i].x)
        xCoordPlot.append("none")
        #y
        yCoordPlot.append(y[i].x)
        yCoordPlot.append(y[i].x + h[i])
        yCoordPlot.append(y[i].x + h[i])
        yCoordPlot.append(y[i].x)
        yCoordPlot.append(y[i].x)
        yCoordPlot.append("none")
fig = go.Figure(go.Scatter(x=xCoordPlot, y=yCoordPlot, fill="toself"))
fig.show()